In [11]:
import re
import csv
import langdetect
from itertools import islice
from youtube_comment_downloader import YoutubeCommentDownloader, SORT_BY_POPULAR

# Inisialisasi downloader
downloader = YoutubeCommentDownloader()

# Fungsi untuk membersihkan komentar (hapus spasi berlebihan, karakter aneh, emoji)
def clean_comment(text):
    # Hapus emoji
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticon
        u"\U0001F300-\U0001F5FF"  # simbol & pictograph
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # bendera (iOS)
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # Hapus "(diedit)" dan spasi berlebihan
    text = re.sub(r'\(diedit\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\n', ' ').replace('\r', '')
    # regex for numbers and special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)  # Hapus spasi berlebihan
    text = text.strip()
    return text

# Fungsi cek apakah komentar berbahasa Inggris
def is_english(text):
    try:
        lang = langdetect.detect(text)
        return lang == 'en'
    except:
        return False

# Ambil komentar dari URL
comments = downloader.get_comments_from_url(
    'https://www.youtube.com/live/fK85SQzm0Z0?si=PyDLiqHhUi6mNmrs',
    sort_by=SORT_BY_POPULAR
)

# List untuk simpan komentar
filtered_comments = []

# Proses komentar
for comment in islice(comments, 15000):
    raw_text = comment.get('text', '')
    cleaned_text = clean_comment(raw_text)

    if is_english(cleaned_text) and cleaned_text:
        filtered_comments.append([cleaned_text])

# Simpan ke file CSV
with open('english_comments.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['comment'])  # Header CSV
    writer.writerows(filtered_comments)

print(f"Selesai! Disimpan {len(filtered_comments)} komentar di 'english_comments.csv'")


Selesai! Disimpan 3019 komentar di 'english_comments.csv'


In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# ===== TAMBAHAN UNTUK SENTIMEN ANALISIS DAN WORDCLOUD =====

from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# lanjut...


# Load data dari CSV
df = pd.read_csv('english_comments.csv')

# Load model sentiment analysis BERT
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Prediksi sentimen
results = sentiment_pipeline(df['comment'].tolist())

# Tambahkan hasil ke dataframe
df['label'] = [r['label'] for r in results]
df['score'] = [r['score'] for r in results]

# Simpan hasil baru ke CSV
df.to_csv('english_comments_with_sentiment.csv', index=False)

print("Sentiment analysis selesai! Disimpan ke 'english_comments_with_sentiment.csv'")

# Buat WordCloud berdasarkan komentar positif
positive_comments = ' '.join(df[df['label'] == 'POSITIVE']['comment'].tolist())

wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(positive_comments)

# Tampilkan WordCloud
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('WordCloud for Positive Comments', fontsize=20)
plt.show()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


model.safetensors:  31%|###1      | 83.9M/268M [00:00<?, ?B/s]